### Larger network with pruned synapses holds more memories than an unpruned smaller network

#### Based on work from [Chechik et al., 1998.](https://www-mitpressjournals-org.proxy.lib.duke.edu/doi/pdf/10.1162/089976698300017124)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy

In [2]:
### we want there to be multiple numbers of neurons, but each network to be pruned to the same amount of synapses (160,000) which is 
### the value when the original 800 neuron network is completely intact

In [3]:
def hopfield_model_synaptic_deletion_min_val(mem, neu, tim, threshold):

    import numpy as np
    import copy
    
    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns
    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns - THIS SEEMS TO BE THE BOTTLENECK
    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        connectivity += np.outer(patterns[m], patterns[m].T)
    for i in range(N):
        connectivity[i][i] = 0
    
    Wij = connectivity / np.sqrt(M)
    z = Wij.copy()
    t = threshold
    
    ## min val deletion
    for i in range(len(z)):
        for j in range(len(z)):
            if np.abs(z[i][j]) > t:
                z[i][j] = z[i][j]
            elif z[i][j] == t:
                z[i][j] = 0
            else:
                z[i][j] = 0
    
    #set the connectivity matrix to the modified/deleted synaptic values
    Wij = z.copy()

    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0].copy()
    n_change = int(X[0].shape[0] * 0.1)
    X[0][1:n_change] = -X[0][1:n_change]
    
    
    #update the model for every timestep based on its sign
    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    #create an array to hold the overlap percentages   
    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    #calculate overlap percentages
    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
            
            
    #get the number of zeros that resulted in the Wij connectivity matrix following this pruning step
    counts = np.unique(Wij, return_counts = True)       
    zeros = counts[1][counts[0] == 0]        
            
    return(m_overlap.take(0, axis = 1)[1], zeros)

In [4]:
def hopfield_model_synaptic_deletion_random(mem, neu, tim, threshold):

    import numpy as np
    import copy
    
    M = mem #number of memories
    N = neu #number of neurons
    timesteps = tim #number of timesteps for the simulation


    #creating patterns
    patterns = np.empty(shape = (M, N))
    for m in range(M):
        patterns[m] = np.random.choice([-1, 1], N)


    #creating the initial connectivity matrix based on a sum of the patterns - THIS SEEMS TO BE THE BOTTLENECK
    connectivity = np.zeros(shape = (N, N))
    for m in range(M):
        connectivity += np.outer(patterns[m], patterns[m].T)
    for i in range(N):
        connectivity[i][i] = 0
    
    Wij = connectivity / np.sqrt(M)
    z = Wij.copy()
    t = threshold
    
    ## random deletion

    indices = np.random.choice(np.arange(z.size), replace=False, size=int(z.size * t))
    z_flat = z.flatten()
    z_flat[indices] = 0
    z = z_flat.reshape((Wij.shape))

    
    #set the connectivity matrix to the modified/deleted synaptic values
    Wij = z.copy()

    #want an initial overlap of 0.8
    X = np.zeros(shape = (timesteps, N))
    X[0] = patterns[0].copy()
    n_change = int(X[0].shape[0] * 0.1)
    X[0][1:n_change] = -X[0][1:n_change]
    
    
    #update the model for every timestep based on its sign
    for t in range(timesteps - 1):
        X[t+1,:] = np.sign(Wij.dot(X[t,:]))

    #create an array to hold the overlap percentages   
    m_overlap = np.empty(shape = (timesteps, M))
    mu = []

    #calculate overlap percentages
    for t in range(timesteps):
        for u in range(M):
            mu = []
            for j in range(N):
                mu = np.append(mu, patterns[u][j] * X[t][j])
            m_overlap[t][u] = ((1/N) * mu.sum())
            
    #get the number of zeros that resulted in the Wij connectivity matrix following this pruning step
    counts = np.unique(Wij, return_counts = True)       
    zeros = counts[1][counts[0] == 0]        
            
    return(m_overlap.take(0, axis = 1)[1], zeros)

In [5]:
%%time

del_levels = np.arange(0, 1.5, .1)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_min_val(mem = int(memory[i]), neu = 900, tim = 2, threshold = 0 + lvl)

Wall time: 1h 15min 54s


In [6]:
nine_min = memory.copy()
nine_min_x1 = x.copy()

In [7]:
nine_min

array([125., 142., 127., 131., 118., 130., 121., 139., 127., 124.,   1.,
         1.,   1.,   1.,   1.])

In [8]:
%%time

del_levels = np.arange(0, 1.5, .1)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_min_val(mem = int(memory[i]), neu = 1000, tim = 2, threshold = 0 + lvl)

Wall time: 1h 30min 8s


In [9]:
thousand_min = memory.copy()
thousand_min_x1 = x.copy()

In [10]:
%%time

del_levels = np.arange(0, 1.5, .1)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_min_val(mem = int(memory[i]), neu = 1100, tim = 2, threshold = 0 + lvl)

Wall time: 1h 44min 24s


In [11]:
eleven_min = memory.copy()
eleven_min_x1 = x.copy()

In [12]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 900, tim = 2, threshold = 0 + lvl)

Wall time: 19min 34s


In [13]:
nine_random = memory.copy()
nine_random_x1 = x.copy()

In [14]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 1000, tim = 2, threshold = 0 + lvl)

Wall time: 28min 21s


In [15]:
thousand_random = memory.copy()
thousand_random_x1 = x.copy()

In [16]:
%%time

del_levels = np.arange(0, 1.05, .05)

x = np.ones(shape = (len(del_levels), 2))
memory = np.zeros(shape = len(del_levels))
i = -1


for lvl in del_levels:
    i = i + 1
    while x[i][0] > .95:
        memory[i] = memory[i] + 1
        x[i] = hopfield_model_synaptic_deletion_random(mem = int(memory[i]), neu = 1100, tim = 2, threshold = 0 + lvl)

Wall time: 38min 3s


In [17]:
eleven_random = memory.copy()
eleven_random_x1 = x.copy()

In [18]:
t = np.concatenate((np.arange(0, 1.5, 0.1), np.arange(0, 1.5, 0.1), np.arange(0, 1.5, 0.1), 
                    np.arange(0, 1.05, .05), np.arange(0, 1.05, .05), np.arange(0, 1.05, .05)))

In [47]:
n_neurons = np.concatenate([np.repeat(900, len(nine_min)), np.repeat(1000, len(thousand_min)), np.repeat('1100', len(eleven_min)), 
                            np.repeat(900, len(nine_random)), np.repeat(1000, len(thousand_random)), np.repeat('1100', len(eleven_random))])
del_type = np.concatenate([np.repeat('min_val', len(nine_min) + len(thousand_min) +len(eleven_min)), 
                           np.repeat('random', len(nine_random) + len(thousand_random) + len(eleven_random))])

In [48]:
nine_min_x = []
for i in range(len(nine_min_x1)):
    nine_min_x = np.append(nine_min_x, nine_min_x1[i][1])
    
thousand_min_x = []
for i in range(len(thousand_min_x1)):
    thousand_min_x = np.append(thousand_min_x, thousand_min_x1[i][1])
    
eleven_min_x = []
for i in range(len(eleven_min_x1)):
    eleven_min_x = np.append(eleven_min_x, eleven_min_x1[i][1])
    
nine_random_x = []
for i in range(len(nine_random_x1)):
    nine_random_x = np.append(nine_random_x, nine_random_x1[i][1])
    
thousand_random_x = []
for i in range(len(thousand_random_x1)):
    thousand_random_x = np.append(thousand_random_x, thousand_random_x1[i][1])
    
eleven_random_x = []
for i in range(len(eleven_random_x1)):
    eleven_random_x = np.append(eleven_random_x, eleven_random_x1[i][1])

In [49]:
n_memories = np.concatenate([nine_min, thousand_min, eleven_min, nine_random, thousand_random, eleven_random])
n_syn_deleted  = np.concatenate([nine_min_x, thousand_min_x, eleven_min_x, nine_random_x, thousand_random_x, eleven_random_x])

In [50]:
df = pd.DataFrame({'threshold':t, 'n_neurons':n_neurons, 'del_type': del_type, 'n_memories':n_memories, 'n_syn_deleted':n_syn_deleted})

In [64]:
df['n_syn'] = (df['n_neurons'].astype('int') * df['n_neurons'].astype('int')) - df['n_syn_deleted']

In [65]:
df.to_csv('C:\\Users\\Minecraft in 4K\\Dropbox\\spr_2020_classes\\quant_neurobio\\final_results_mean_tidy_fig2.csv')

In [74]:
df.head()

,threshold,n_neurons,del_type,n_memories,n_syn_deleted,n_syn
0,0.0,900,min_val,125.0,900.0,809100.0
1,0.1,900,min_val,142.0,55336.0,754664.0
2,0.2,900,min_val,127.0,115530.0,694470.0
3,0.3,900,min_val,131.0,221830.0,588170.0
4,0.4,900,min_val,118.0,286466.0,523534.0


In [66]:
df_800 = pd.read_csv('C:\\Users\\Minecraft in 4K\\Dropbox\\spr_2020_classes\\quant_neurobio\\final_results_mean_tidy.csv')

In [68]:
df_800['n_neurons'] = np.repeat(800, len(df_800))

In [75]:
df_800['n_syn'] = (df_800['n_neurons'].astype('int') * df_800['n_neurons'].astype('int')) - df_800['n_syn_deleted']

In [77]:
df_800.rename(columns = {'n_mem':'n_memories'}, inplace = True)

In [83]:
df_800.drop('del_level', axis =1, inplace = True)

In [87]:
df_final = df.append(df_800)

In [91]:
np.unique(df_final['del_type'], return_counts = True)

(array(['clipping', 'compressed', 'min_val', 'random'], dtype=object),
 array([15, 15, 60, 84], dtype=int64))

In [92]:
df_final.to_csv('C:\\Users\\Minecraft in 4K\\Dropbox\\spr_2020_classes\\quant_neurobio\\final_results_mean_tidy_fig2.csv')